In [99]:
import pandas as pd
import matplotlib.pyplot as plt
import networkx as nx
import math
import re

In [100]:
# === Leitura e pré-processamento dos dados ===
def carregar_dados(path):
    df = pd.read_csv(path, encoding='latin-1')
    colunas_remover = ['Product Image', 'Product Description', 'Order Zipcode', 'Customer Password', 'Customer Email']
    df.drop(columns=colunas_remover, inplace=True)
    df['order date (DateOrders)'] = pd.to_datetime(df['order date (DateOrders)'], errors='coerce')
    df['Date'] = df['order date (DateOrders)'].dt.date
    return df

In [101]:
# === Utilitários para pesos e mapeamentos ===
def add_weight_mean(df, group_col, new_col):
    df[new_col] = df.groupby(group_col)['Weight'].transform('mean')

# def create_mapping(unique_values, prefix='', start=1):
#     return {val: f"{prefix}{i+start}" if prefix else i+start for i, val in enumerate(unique_values)}

def create_weight_dict(df, keys_cols, values_col):
    keys = [df[col] for col in keys_cols]
    return dict(zip(zip(*keys), df[values_col]))


In [102]:
# === Algoritmo de Bellman-Ford ===
def bellman_ford(N, A, c, s):
    d = {j: float('inf') for j in N}
    p = {j: None for j in N}
    d[s] = 0
    p[s] = 0

    for _ in range(len(N) - 1):
        for (i, j) in A:
            if d[i] != float('inf') and d[j] > d[i] + c[(i, j)]:
                d[j] = d[i] + c[(i, j)]
                p[j] = i

    # Verificação de ciclo negativo
    for (i, j) in A:
        if d[i] != float('inf') and d[j] > d[i] + c[(i, j)]:
            raise ValueError("Ciclo negativo detectado")

    return d, p

In [103]:
def reconstruir_caminho(p, destino):
    caminho = []
    atual = destino
    while atual in p and p[atual] is not None:
        caminho.append(atual)
        atual = p[atual]
    caminho.append(atual)
    caminho.reverse()
    return caminho

def calcular_custo_caminho(caminho, pesos):
    return sum(pesos.get((caminho[i], caminho[i+1]), 0) for i in range(len(caminho) - 1))

In [104]:
def r_cycle(arcs_neg):
    G = nx.DiGraph()
    G.add_weighted_edges_from([(u, v, w) for (u, v), w in arcs_neg.items()])

    while True:
        try:
            cycle = nx.find_cycle(G, orientation='original')
            u, v, *_ = cycle[0]
            G.remove_edge(u, v)

        except nx.NetworkXNoCycle:
            break  

    return {(u, v): G[u][v]['weight'] for u, v in G.edges()}

In [105]:
# === Processamento principal ===
def processar(df, date_range):
    results = {}

    for date in date_range:
        request = df[df['Date'] == date.date()].copy()

        # Cálculo de peso
        request['Weight'] = (
            request['Order Item Quantity'] * request['Order Profit Per Order']
        ) / (
            request['Days for shipping (real)'] + 
            request['Late_delivery_risk'] * request['Days for shipment (scheduled)']
        )

        # Médias de peso por agrupamento
        groupings = {
            'Customer City': 'W_city_state_cus',      # Calcula o peso Cidade-Estado       (Customer)
            'Customer State': 'W_state_country_cus',  # Calcula o peso Estado-País         (Customer)
            'Order Country': 'W_country_country',     # Calcula o peso entre Países        (Customer->Order) 
            'Order State': 'W_country_state_ord',     # Calcula o peso entre País-Estado   (Order)
            'Order City': 'W_state_city_ord',         # Calcula o peso entre Estado-Cidade (Order)
        }

        for group, new_column in groupings.items():   # Calcula as médias com base nos grupos
            add_weight_mean(request, group, new_column)
            
        # Criação dos arcos
        arcs = {}
        arcs |= create_weight_dict(request, ['Customer City', 'Customer State'], 'W_city_state_cus')
        arcs |= create_weight_dict(request, ['Customer State', 'Customer Country'], 'W_state_country_cus')
        arcs |= create_weight_dict(request, ['Customer Country', 'Order Country'], 'W_country_country')
        arcs |= create_weight_dict(request, ['Order Country', 'Order State'], 'W_country_state_ord')
        arcs |= create_weight_dict(request, ['Order State', 'Order City'], 'W_state_city_ord')


        arcs_neg = {k: -v for k, v in arcs.items()}
        arcs_neg = r_cycle(arcs_neg)
        edges = [(u, v) for (u, v) in arcs_neg if pd.notna(u) and pd.notna(v)]
        nodes = set(u for u, v in edges) | set(v for u, v in edges)

        greatest_path = {}
        for city in request['Customer City'].unique():
            d, p = bellman_ford(nodes, edges, arcs_neg, city)
            destinos_validos = [n for n in p if n in request['Order City'].values]

            melhor_caminho, melhor_custo = [], float('inf')
            for destino in destinos_validos:
                caminho = reconstruir_caminho(p, destino)
                custo = calcular_custo_caminho(caminho, arcs_neg)
                if custo < melhor_custo:
                    melhor_caminho, melhor_custo = caminho, custo

            greatest_path[city] = [melhor_caminho, melhor_custo]

        # Filtra caminhos cujo custo é finito
        valid_paths = {k: v for k, v in greatest_path.items() if math.isfinite(v[1])}

        # Verifica se há algum válido
        optimal_sol = min(valid_paths, key=lambda k: valid_paths[k][1]) if valid_paths else None

        # cidade_otima = min(greatest_path, key=lambda k: greatest_path[k][1])
        results[date] = greatest_path[optimal_sol] if optimal_sol else None

    return results

In [108]:
# === Execução === (30 dias)
df = carregar_dados('dataco.csv')
intervalo_datas = pd.date_range(start='2015-01-01', periods=30)
results = processar(df, intervalo_datas)

# Filtra resultados válidos (que não são None)
results = {k: v for k, v in results.items() if v is not None and math.isfinite(v[1])}
[results.update({k: [v[0], abs(v[1])]}) for k, v in results.items() if isinstance(v[1], (int, float))]

if results:
    best_day = max(results, key=lambda k: results[k][1])
    print(best_day)
    print(results[best_day])
else:
    print("Nenhum resultado válido encontrado.")


2015-01-23 00:00:00
[[0, 'Williamsport', 'PA', 'EE. UU.', 'Brasil', 'São Paulo', 'São José dos Campos'], 958.0162721607292]


In [ ]:
# === Execução === (6 meses)
intervalo_datas = pd.date_range(start='2015-01-01', periods=180)
results = processar(df, intervalo_datas)

# Filtra resultados válidos (que não são None)
results = {k: v for k, v in results.items() if v is not None and math.isfinite(v[1])}
[results.update({k: [v[0], abs(v[1])]}) for k, v in results.items() if isinstance(v[1], (int, float))]

if results:
    best_day = max(results, key=lambda k: results[k][1])
    print(best_day)
    print(results[best_day])
else:
    print("Nenhum resultado válido encontrado.")


2015-05-01 00:00:00
[[0, 'Baltimore', 'MD', 'EE. UU.', 'México', 'Coahuila', 'Monclova'], 1302.7648135581633]


In [ ]:
# === Execução === (1 ano)
intervalo_datas = pd.date_range(start='2015-01-01', periods=360)
results = processar(df, intervalo_datas)

# Filtra resultados válidos (que não são None)
results = {k: v for k, v in results.items() if v is not None and math.isfinite(v[1])}
[results.update({k: [v[0], abs(v[1])]}) for k, v in results.items() if isinstance(v[1], (int, float))]

if results:
    best_day = max(results, key=lambda k: results[k][1])
    print(best_day)
    print(results[best_day])
else:
    print("Nenhum resultado válido encontrado.")


2015-08-29 00:00:00
[[0, 'Albuquerque', 'NM', 'EE. UU.', 'Noruega', 'Oslo'], 1478.2000732000001]


In [ ]:
for date in pd.date_range(start='2015-01-23', periods=1):
    um_mes_df = df[df['Date'] == date.date()]

melhor_um_mes = um_mes_df[um_mes_df['Customer City'] == 'Williamsport']

In [180]:
for date in pd.date_range(start='2015-05-01', periods=1):
    seis_meses_df = df[df['Date'] == date.date()]

melhor_seis_meses = seis_meses_df[seis_meses_df['Customer City'] == 'Baltimore']

In [181]:
for date in pd.date_range(start='2015-08-29', periods=1):
    um_ano_df = df[df['Date'] == date.date()]

melhor_um_ano = um_ano_df[um_ano_df['Customer City'] == 'Albuquerque']

In [186]:
melhor_um_mes.iloc[:,30:]

,Order Item Discount Rate,Order Item Id,Order Item Product Price,Order Item Profit Ratio,Order Item Quantity,Sales,Order Item Total,Order Profit Per Order,Order Region,Order State,Order Status,Product Card Id,Product Category Id,Product Name,Product Price,Product Status,shipping date (DateOrders),Shipping Mode,Date
164959,0.06,3827,99.989998,0.45,5,499.950012,472.450012,212.600006,Central America,Francisco Morazán,COMPLETE,191,9,Nike Men's Free 5.0+ Running Shoe,99.989998,0,1/26/2015 8:22,Standard Class,2015-01-23


In [199]:
melhor_seis_meses.iloc[:,30:]

,Order Item Discount Rate,Order Item Id,Order Item Product Price,Order Item Profit Ratio,Order Item Quantity,Sales,Order Item Total,Order Profit Per Order,Order Region,Order State,Order Status,Product Card Id,Product Category Id,Product Name,Product Price,Product Status,shipping date (DateOrders),Shipping Mode,Date
61953,0.15,20653,199.990005,0.10,1,199.990005,169.990005,17.000000,South America,Córdoba,PROCESSING,1073,48,Pelican Sunstream 100 Kayak,199.990005,0,5/2/2015 3:56,Same Day,2015-05-01
71651,0.16,20654,59.990002,0.49,4,239.960007,201.570007,98.769997,South America,Córdoba,PROCESSING,365,17,Perfect Fitness Perfect Rip Deck,59.990002,0,5/2/2015 3:56,Same Day,2015-05-01
71691,0.01,20655,59.990002,0.49,5,299.950012,296.950012,145.509995,South America,Córdoba,PROCESSING,365,17,Perfect Fitness Perfect Rip Deck,59.990002,0,5/2/2015 3:56,Same Day,2015-05-01
71692,0.02,20652,59.990002,0.31,5,299.950012,293.950012,92.010002,South America,Córdoba,PROCESSING,365,17,Perfect Fitness Perfect Rip Deck,59.990002,0,5/2/2015 3:56,Same Day,2015-05-01


In [200]:
melhor_um_ano.iloc[:,30:]

,Order Item Discount Rate,Order Item Id,Order Item Product Price,Order Item Profit Ratio,Order Item Quantity,Sales,Order Item Total,Order Profit Per Order,Order Region,Order State,Order Status,Product Card Id,Product Category Id,Product Name,Product Price,Product Status,shipping date (DateOrders),Shipping Mode,Date
117702,0.12,41239,59.990002,0.28,5,299.950012,263.959992,73.910004,Northern Europe,Oslo,PROCESSING,365,17,Perfect Fitness Perfect Rip Deck,59.990002,0,8/30/2015 11:50,Same Day,2015-08-29
